In [2]:
import pandas as pd
import numpy as np


In [3]:
original_df_dict = pd.read_excel('../Data/Original/企業名辞書v2023_1.xlsx', sheet_name=None, engine='openpyxl')


In [4]:
master_df = original_df_dict['企業名辞書v2023_1'].copy()
col_dict = dict(zip(master_df.columns, master_df.iloc[0, :].copy()))
needed_col_list = ['comp_id', 'comp_name', 'comp_code', 'post_comp_id', 'parent_compid', 'parent_comp_name']

master_df.columns = [col_dict[col] for col in master_df.columns]
master_df = master_df[needed_col_list].iloc[1:, :].replace('\\N', np.nan)



In [5]:
# 親子関係
parent_df = master_df[['comp_id', 'parent_compid', 'parent_comp_name']].drop_duplicates().dropna()
parent_df


,comp_id,parent_compid,parent_comp_name
10,1007873,95.0,三井化学
25,65532,1009842.0,アークランドサカモト
26,1003085,1009842.0,アークランドサカモト
34,76043,1009842.0,アークランドサカモト
52,74418,205.0,川崎重工業
...,...,...,...
29132,67406,65935.0,若井ホ―ルデイングス
29160,1002813,1310.0,日清オイリオグループ
29174,1022593,2594.0,ワコールホールディングス
29208,65632,1003199.0,綿半ホールディングス


In [6]:
# 法人格込みの企業名
id_name_code_df = master_df[['comp_id', 'comp_name', 'comp_code']].drop_duplicates().astype(str)
conv_dict = {'KB':'株式会社', 
             'YG':'有限会社', 
             'GD':'合同会社', 
             'SG':'相互会社', 
             'GS':'合資会社', 
             'GM':'合名会社'}
id_name_code_df['comp_kind'] = id_name_code_df['comp_code'].str[:2].replace(conv_dict)
conditions = [
    (id_name_code_df['comp_code'].str[-1]==i)&~(id_name_code_df['comp_name'].str.contains('会社'))
    for i in ['1', '2']
]
choice = [id_name_code_df['comp_kind']+id_name_code_df['comp_name'], 
          id_name_code_df['comp_name']+id_name_code_df['comp_kind']]
id_name_code_df['full_comp_name'] = np.select(conditions, choice, default=id_name_code_df['comp_name'])
id_name_code_df = id_name_code_df[['comp_id', 'full_comp_name']].drop_duplicates()
id_name_code_df


,comp_id,full_comp_name
1,70519,アーカイラス・エンジニアリング合資会社
2,70520,株式会社アーキス
3,1017291,アーキテクツ・スタジオ・ジャパン株式会社
4,80309,ＴＥＭＰＯ ＮＥＴＷＯＲＫ株式会社
5,80310,レンタル保証システム株式会社
...,...,...
29267,72731,株式会社ワントゥーテンロボティクス
29268,71933,株式会社ワントゥーテンデザイン
29269,71526,株式会社ワン・トゥー・テン・デザイン
29270,71932,有限会社ワン・トゥー・テン・デザイン


In [7]:
# 新旧関係
before_after_df = pd.merge(master_df[['comp_id', 'post_comp_id']].drop_duplicates().rename(columns={'comp_id':'1st_comp_id'}),
                           master_df[['comp_id', 'post_comp_id']].drop_duplicates().rename(columns={'comp_id':'2nd_comp_id', 'post_comp_id':'3rd_comp_id'}), 
                           left_on='post_comp_id', right_on='2nd_comp_id', how='left')\
                           [['1st_comp_id', '2nd_comp_id', '3rd_comp_id']]
before_after_df = pd.merge(before_after_df,
                           master_df[['comp_id', 'post_comp_id']].drop_duplicates().rename(columns={'comp_id':'3rd_comp_id', 'post_comp_id':'4th_comp_id'}), 
                           on='3rd_comp_id', how='left')\
                           [['1st_comp_id', '2nd_comp_id', '3rd_comp_id', '4th_comp_id']]
for i in range(4, 11+1):
    before_after_df = pd.merge(before_after_df,
                               master_df[['comp_id', 'post_comp_id']].drop_duplicates().rename(columns={'comp_id':f'{i}th_comp_id', 
                                                                                                        'post_comp_id':f'{i+1}th_comp_id'}), 
                               on=f'{i}th_comp_id', how='left')
before_after_df['after_id'] = before_after_df['11th_comp_id'].copy()
for i in range(10, 4-1, -1):
    before_after_df['after_id'] = before_after_df['after_id'].fillna(before_after_df[f'{i}th_comp_id'])
    if i == 4:
        for j in ['3rd', '2nd', '1st']:
            before_after_df['after_id'] = before_after_df['after_id'].fillna(before_after_df[f'{j}_comp_id'])
before_after_df = before_after_df[['1st_comp_id', 'after_id']]\
                                 .rename(columns={'1st_comp_id':'before_id'})\
                                 .astype(np.int64)
before_after_df


,before_id,after_id
0,70519,70519
1,70520,70520
2,1017291,1017291
3,80309,1017291
4,80310,1017291
...,...,...
29266,72731,71934
29267,71933,71934
29268,71526,71934
29269,71932,71934


In [9]:
# 新旧親子関係
before_parent_df = pd.merge(before_after_df, parent_df, 
                            left_on='after_id', right_on='comp_id', how='left')
# before_parent_df['after_parentid'] = before_parent_df['parent_compid'].fillna(before_parent_df['after_id']).astype(np.int64)
# before_parent_df = before_parent_df[['before_id', 'after_parentid']].rename(columns={'after_parentid':'after_id'})
before_parent_df = before_parent_df[['before_id', 'after_id']]
before_parent_df


,before_id,after_id
0,70519,70519
1,70520,70520
2,1017291,1017291
3,80309,1017291
4,80310,1017291
...,...,...
29266,72731,71934
29267,71933,71934
29268,71526,71934
29269,71932,71934


In [17]:
before_after_name_df = pd.merge(before_parent_df.astype(str), id_name_code_df, 
                                left_on='before_id', right_on='comp_id', how='left')\
                                [['before_id', 'after_id', 'full_comp_name']]\
                                .rename(columns={'full_comp_name':'before_name'})
before_after_name_df = pd.merge(before_after_name_df.astype(str), id_name_code_df, 
                                left_on='after_id', right_on='comp_id', how='left')\
                                [['before_name', 'full_comp_name']]\
                                .rename(columns={'full_comp_name':'after_name'})
before_after_name_df


,before_name,after_name
0,アーカイラス・エンジニアリング合資会社,アーカイラス・エンジニアリング合資会社
1,株式会社アーキス,株式会社アーキス
2,アーキテクツ・スタジオ・ジャパン株式会社,アーキテクツ・スタジオ・ジャパン株式会社
3,ＴＥＭＰＯ ＮＥＴＷＯＲＫ株式会社,アーキテクツ・スタジオ・ジャパン株式会社
4,レンタル保証システム株式会社,アーキテクツ・スタジオ・ジャパン株式会社
...,...,...
29266,株式会社ワントゥーテンロボティクス,株式会社ワントゥーテン
29267,株式会社ワントゥーテンデザイン,株式会社ワントゥーテン
29268,株式会社ワン・トゥー・テン・デザイン,株式会社ワントゥーテン
29269,有限会社ワン・トゥー・テン・デザイン,株式会社ワントゥーテン


In [18]:
old_new_letter_df = pd.read_html(
    'https://www.asahi-net.or.jp/~ax2s-kmtn/ref/old_chara.html',
    encoding="utf8"
)
df_tmp_1 = old_new_letter_df[1].iloc[:,0:6]
df_tmp_2 = old_new_letter_df[1].iloc[:,6:12]
df_tmp_3 = old_new_letter_df[1].iloc[:,12:18]
df_tmp_4 = old_new_letter_df[2].iloc[:,0:6]
df_tmp_5 = old_new_letter_df[2].iloc[:,6:12]
df_tmp_6 = old_new_letter_df[2].iloc[:,12:18]

# 列名を統一
new_columns = [
		'new_jis',
    'new_unicode',
    'new_jikei',
    'old_jikei',
    'old_jis',
    'old_unicode',
]

df_tmp_1.columns = new_columns
df_tmp_2.columns = new_columns
df_tmp_3.columns = new_columns
df_tmp_4.columns = new_columns
df_tmp_5.columns = new_columns
df_tmp_6.columns = new_columns

# 表を縦い合体する
new_df = pd.concat([df_tmp_1, df_tmp_2], axis=0)
new_df = pd.concat([new_df, df_tmp_3], axis=0)
new_df = pd.concat([new_df, df_tmp_4], axis=0)
new_df = pd.concat([new_df, df_tmp_5], axis=0)
new_df = pd.concat([new_df, df_tmp_6], axis=0)

# データの入っていない行を削除
new_df = new_df.dropna(how='all')

# インデックスを振り直す
new_df = new_df.reset_index(drop=True)

# 旧字体と新字体の辞書を作成
old_kanji = list(new_df['old_jikei'])
new_kanji = list(new_df['new_jikei'])
jitai_dict = dict(zip(old_kanji, new_kanji))

# 辞書を検索して文字列を置き換える関数を作成
def kyujitai_to_shinjitai(text):
    encoded_text = text.translate(str.maketrans(jitai_dict))
    return encoded_text


In [19]:
before_after_name_df['before_name'] = before_after_name_df['before_name'].str.translate(str.maketrans(jitai_dict))
before_after_name_df['after_name'] = before_after_name_df['after_name'].str.translate(str.maketrans(jitai_dict))
before_after_name_df


,before_name,after_name
0,アーカイラス・エンジニアリング合資会社,アーカイラス・エンジニアリング合資会社
1,株式会社アーキス,株式会社アーキス
2,アーキテクツ・スタジオ・ジャパン株式会社,アーキテクツ・スタジオ・ジャパン株式会社
3,ＴＥＭＰＯ ＮＥＴＷＯＲＫ株式会社,アーキテクツ・スタジオ・ジャパン株式会社
4,レンタル保証システム株式会社,アーキテクツ・スタジオ・ジャパン株式会社
...,...,...
29266,株式会社ワントゥーテンロボティクス,株式会社ワントゥーテン
29267,株式会社ワントゥーテンデザイン,株式会社ワントゥーテン
29268,株式会社ワン・トゥー・テン・デザイン,株式会社ワントゥーテン
29269,有限会社ワン・トゥー・テン・デザイン,株式会社ワントゥーテン


In [20]:
before_after_name_df.to_csv('../Data/Dealed/company_master.csv', sep=',', encoding='utf-8', index=False)
